In [24]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
pd.set_option('display.max_columns', None)

<IPython.core.display.Javascript object>

In [26]:
%%R

library(car)


In [27]:
%%R

registration <- read.csv('RER_llistat_explotacions_catalanes.csv', fileEncoding = 'ISO-8859-1')
head(registration)

      MO      CODI.REGA ESTAT.EXPLOTACIÓ DATA.CANVI.ESTAT.EXPLOTACIÓ
1 0010AA ES250010030750           Activa                  21/01/1997
2 0010AA ES250010030750           Activa                  21/01/1997
3 0010AA ES250010030750           Activa                  21/01/1997
4 0010AA ES250010030750           Activa                  21/01/1997
5 0010AB ES250010031100           Activa                  21/04/1997
6 0010AC ES250010031700           Activa                  01/12/1997
           NOM.EXPLOTACIO  ADREÇA.EXPLOTACIÓ CODI.POSTAL.EXPLOTACIO
1 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
2 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
3 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
4 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
5            CASA GONELLA            BOIXOLS                  25651
6        SORT DE L'AGUSTÍ ABELLA DE LA CONCA                  25651
  SERVEI.TERRITORIAL..EXPLOTACIÓ PROVINCI

### Data Cleaning ###

In [28]:
%%R

# Ditch rows with SISTEMA.PRODUCTIU as blank
registration <- registration[registration$SISTEMA.PRODUCTIU != '',]

In [29]:
%%R

# Ditch rows where TOTAL URM is 0
registration <- registration[registration$TOTAL.URM != 0,]

# Ditch rows where TOTAL NITROGEN is 0
registration <- registration[registration$TOTAL.NITROGEN != 0,]

In [30]:
%%R

# Add a column "Is Intensive", which is 1 if SISTEMA.PRODUCTIU is "intensiu" and 0 otherwise
registration$Is.Intensive <- ifelse(registration$SISTEMA.PRODUCTIU == 'Intensiu', 1, 0)

In [31]:
%%R

# From 'DATA.CANVI.ESTAT.EXPLOTACIÓ' column, extract the year, and put it in a new column 'YEAR'
registration$YEAR <- as.numeric(format(as.Date(registration$DATA.CANVI.ESTAT.EXPLOTACIÓ, format="%d/%m/%Y"), "%Y"))

In [32]:
%%R

# Add a column "Is Pig", which is 1 if ESPÈCIE == 'PORCÍ', and 0 otherwise
registration$IsPig <- ifelse(registration$ESPÈCIE == 'Porcí', 1, 0)

# Add a column "Is Cattle", which is 1 if ESPÈCIE == 'BOVÍ', and 0 otherwise
registration$IsCattle <- ifelse(registration$ESPÈCIE == 'Boví', 1, 0)

# Add a column "Is Poultry", which is 1 if ESPÈCIE == 'Gallines i pollastres', and 0 otherwise
registration$IsPoultry <- ifelse(registration$ESPÈCIE == 'Gallines i pollastres', 1, 0)

# Add a column "Is Sheep", which is 1 if ESPÈCIE == 'Oví', and 0 otherwise
registration$IsSheep <- ifelse(registration$ESPÈCIE == 'Oví', 1, 0)

### Regression ###

In [44]:
%%R

# Run a regression to predict nitrogen based on the following variables:
# - Is Intensive
# - Is Pig
# - Is Cattle
# - Is Poultry
# - Is Sheep
# - YEAR
# - TOTAL.URM

model <- lm(TOTAL.NITROGEN ~ Is.Intensive + IsPig + IsCattle + IsPoultry + IsSheep + YEAR + TOTAL.URM, data=registration)
summary(model)


Call:
lm(formula = TOTAL.NITROGEN ~ Is.Intensive + IsPig + IsCattle + 
    IsPoultry + IsSheep + YEAR + TOTAL.URM, data = registration)

Residuals:
   Min     1Q Median     3Q    Max 
-56934   -461    -21    182  63370 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  15826.3918  3244.3554   4.878 1.08e-06 ***
Is.Intensive  -599.8550    39.0139 -15.375  < 2e-16 ***
IsPig          395.6880    52.8952   7.481 7.65e-14 ***
IsCattle     -2450.7502    46.9188 -52.234  < 2e-16 ***
IsPoultry      796.1509    70.8486  11.237  < 2e-16 ***
IsSheep      -1054.7211    70.0169 -15.064  < 2e-16 ***
YEAR            -7.5821     1.6152  -4.694 2.69e-06 ***
TOTAL.URM       58.3754     0.1362 428.583  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2474 on 24350 degrees of freedom
Multiple R-squared:  0.9129,	Adjusted R-squared:  0.9129 
F-statistic: 3.645e+04 on 7 and 24350 DF,  p-value: < 2.2e-16



In [34]:
%%R

head(registration)

      MO      CODI.REGA ESTAT.EXPLOTACIÓ DATA.CANVI.ESTAT.EXPLOTACIÓ
1 0010AA ES250010030750           Activa                  21/01/1997
2 0010AA ES250010030750           Activa                  21/01/1997
3 0010AA ES250010030750           Activa                  21/01/1997
4 0010AA ES250010030750           Activa                  21/01/1997
5 0010AB ES250010031100           Activa                  21/04/1997
6 0010AC ES250010031700           Activa                  01/12/1997
           NOM.EXPLOTACIO  ADREÇA.EXPLOTACIÓ CODI.POSTAL.EXPLOTACIO
1 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
2 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
3 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
4 CASA JULIU - JORDI GASA        MASIA JULIU                  25651
5            CASA GONELLA            BOIXOLS                  25651
6        SORT DE L'AGUSTÍ ABELLA DE LA CONCA                  25651
  SERVEI.TERRITORIAL..EXPLOTACIÓ PROVINCI

## GEO ANALYSIS ##

In [45]:
%%R
library(dplyr)

# Filter the dataset to include only rows where ESPÈCIE == 'Porcí'
pig_farms <- registration[registration$ESPÈCIE == 'Porcí',]

# Group by PROVINCIA.EXPLOTACIÓ and count the number of pig farms in each province
pig_farms_by_province <- pig_farms %>%
  group_by(PROVINCIA.EXPLOTACIÓ) %>%
  summarise(NUM_PIG_FARMS = n()) %>%
  arrange(desc(NUM_PIG_FARMS))

# Print the results
print(pig_farms_by_province)

# A tibble: 4 × 2
  PROVINCIA.EXPLOTACIÓ NUM_PIG_FARMS
  <chr>                        <int>
1 Lleida                        2793
2 Barcelona                     1398
3 Girona                         768
4 Tarragona                      368


In [48]:
%%R

# Group by MUNICIPI.EXPLOTACIÓ and count the number of pig farms in each municipality
pig_farms_by_municipality <- pig_farms %>%
  group_by(MUNICIPI.EXPLOTACIÓ) %>%
  summarise(NUM_PIG_FARMS = n()) %>%
  arrange(desc(NUM_PIG_FARMS))

# Print the results
print(pig_farms_by_municipality)

# A tibble: 577 × 2
   MUNICIPI.EXPLOTACIÓ           NUM_PIG_FARMS
   <chr>                                 <int>
 1 Almenar                                 111
 2 Alcarràs                                 96
 3 Gurb                                     79
 4 Artesa de Segre                          77
 5 Lleida                                   75
 6 Juneda                                   59
 7 Agramunt                                 58
 8 Montgai                                  54
 9 Gimenells i el Pla de la Font            51
10 Torregrossa                              49
# ℹ 567 more rows
# ℹ Use `print(n = ...)` to see more rows


In [50]:
%%R

# Group by CODI.POSTAL.EXPLOTACIO and count the number of pig farms in each postal code
pig_farms_by_postal_code <- pig_farms %>%
  group_by(CODI.POSTAL.EXPLOTACIO) %>%
  summarise(NUM_PIG_FARMS = n()) %>%
  arrange(desc(NUM_PIG_FARMS))

# Print the results
print(pig_farms_by_postal_code)

# A tibble: 519 × 2
   CODI.POSTAL.EXPLOTACIO NUM_PIG_FARMS
                    <int>         <int>
 1                   8519           126
 2                  25126           111
 3                  25180            96
 4                  25212            81
 5                   8503            79
 6                  25730            76
 7                  25749            62
 8                  25430            59
 9                  25310            57
10                  25286            55
# ℹ 509 more rows
# ℹ Use `print(n = ...)` to see more rows
